In [10]:
from  PIL import Image
img = Image.open('bw.jpg').convert('L')   

In [14]:
#img

In [12]:
import numpy as np
print(type(img), np.min(img), np.max(img))

<class 'PIL.Image.Image'> 0 255


In [13]:
img = np.array(img) 
img.shape

(397, 600)

In [9]:
img

array([[29, 31, 33, ..., 20, 20, 20],
       [30, 31, 33, ..., 20, 20, 20],
       [32, 32, 34, ..., 20, 20, 20],
       ...,
       [34, 33, 33, ..., 81, 80, 78],
       [29, 29, 28, ..., 77, 74, 71],
       [27, 27, 27, ..., 73, 69, 66]], dtype=uint8)

In [15]:
from  PIL import Image
img = Image.open('color.jpg')

In [17]:
#img

In [18]:
import numpy as np
print(type(img), np.min(img), np.max(img))

<class 'PIL.JpegImagePlugin.JpegImageFile'> 0 255


In [19]:
img = np.array(img) 
img.shape

(822, 600, 3)

## IM2COL

In [23]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col
    


In [24]:
import numpy as np
x1 = np.random.rand(1,3,7,7)
x1.shape

(1, 3, 7, 7)

In [25]:
col1 = im2col(x1, 5, 5,stride=1, pad = 0)
col1.shape

(9, 75)

## 使用ANN 辨識手寫數字

In [26]:
import keras

C:\ProgramData\Anaconda3_2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [27]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Activation, Dense

In [33]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [34]:
x_train.shape

(60000, 28, 28)

In [35]:
x_test.shape

(10000, 28, 28)

In [36]:
x_train = x_train.reshape(60000, 784)
x_test  = x_test.reshape(10000, 784)

In [37]:
x_train.shape

(60000, 784)

In [38]:
x_test.shape

(10000, 784)

In [39]:
x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')
x_train /= 255
x_test  /= 255

In [40]:
n_classes  = 10 
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test  = keras.utils.to_categorical(y_test, n_classes)

In [41]:
n_hidden_1 = 256 
n_hidden_2 = 256 
n_input    = 784 
n_classes  = 10 

training_epochs = 15
batch_size = 100

In [42]:
model = keras.Sequential()
model.add(Dense(n_hidden_1, activation='relu', input_shape=(n_input,)))
model.add(Dense(n_hidden_2, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

In [43]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [44]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=training_epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 3s 53us/step - loss: 0.2395 - acc: 0.9291 - val_loss: 0.1064 - val_acc: 0.9690
Epoch 2/15
60000/60000 [==============================] - 3s 47us/step - loss: 0.0920 - acc: 0.9722 - val_loss: 0.0890 - val_acc: 0.9720
Epoch 3/15
60000/60000 [==============================] - 3s 46us/step - loss: 0.0602 - acc: 0.9815 - val_loss: 0.0708 - val_acc: 0.9764
Epoch 4/15
60000/60000 [==============================] - 3s 45us/step - loss: 0.0441 - acc: 0.9860 - val_loss: 0.0709 - val_acc: 0.9790
Epoch 5/15
60000/60000 [==============================] - 3s 44us/step - loss: 0.0327 - acc: 0.9893 - val_loss: 0.0778 - val_acc: 0.9774
Epoch 6/15
60000/60000 [==============================] - 3s 46us/step - loss: 0.0243 - acc: 0.9920 - val_loss: 0.0618 - val_acc: 0.9809
Epoch 7/15
60000/60000 [==============================] - 3s 48us/step - loss: 0.0217 - acc: 0.9927 - val_loss: 0.0802 - val_acc

## 使用CNN 辨識手寫數字

In [45]:
from keras.layers import Activation, Dense, Flatten, Conv2D, MaxPooling2D

### 資料前處理

In [46]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [47]:
x_train = x_train.reshape(60000, 28,28,1)
x_test = x_test.reshape(10000, 28,28,1)
x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')
x_train /= 255
x_test  /= 255

In [48]:
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test  = keras.utils.to_categorical(y_test, n_classes)

### 設定網路參數

In [49]:
n_hidden_1 = 256 
n_classes  = 10 

training_epochs = 15
batch_size = 100

### 建立卷積神經網路

In [50]:
model = keras.Sequential()
model.add(Conv2D(
            filters=16,
            kernel_size = (3,3),
            padding = 'same',
            input_shape = (28,28,1), 
            activation = 'relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

model.add(Dense(n_hidden_1, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

In [51]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### 訓練神經網路

In [52]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=15,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 22s 366us/step - loss: 0.2094 - acc: 0.9400 - val_loss: 0.0830 - val_acc: 0.9740
Epoch 2/15
60000/60000 [==============================] - 22s 362us/step - loss: 0.0676 - acc: 0.9797 - val_loss: 0.0604 - val_acc: 0.9820
Epoch 3/15
60000/60000 [==============================] - 22s 369us/step - loss: 0.0430 - acc: 0.9869 - val_loss: 0.0446 - val_acc: 0.9855
Epoch 4/15
60000/60000 [==============================] - 22s 362us/step - loss: 0.0310 - acc: 0.9910 - val_loss: 0.0529 - val_acc: 0.9823
Epoch 5/15
60000/60000 [==============================] - 22s 364us/step - loss: 0.0227 - acc: 0.9932 - val_loss: 0.0462 - val_acc: 0.9847
Epoch 6/15
60000/60000 [==============================] - 22s 362us/step - loss: 0.0165 - acc: 0.9950 - val_loss: 0.0463 - val_acc: 0.9866
Epoch 7/15
60000/60000 [==============================] - 22s 363us/step - loss: 0.0128 - acc: 0.9961 - val_loss: 0.

In [53]:
model.save('OCR.hd5')

In [56]:
for ele in model.get_weights():
    print(ele.shape)

(3, 3, 1, 16)
(16,)
(3136, 256)
(256,)
(256, 10)
(10,)


In [58]:
14 * 14 * 16

3136

In [57]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               803072    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                2570      
Total params: 805,802
Trainable params: 805,802
Non-trainable params: 0
_________________________________________________________________


In [59]:
from keras.models import load_model
model2 = load_model('OCR.hd5')


In [61]:
#model2.predict(x_test)

In [63]:
from PIL import Image
import numpy
img  = Image.open('4.png').convert('L')
img  = numpy.asarray(img)

In [67]:
img2 = (255 - img) / 255

In [68]:
img2.reshape(28,28,1)

ValueError: cannot reshape array of size 64 into shape (28,28,1)